In [5]:
from epitator.annotator import AnnoDoc
from epitator.geoname_annotator import GeonameAnnotator
from epitator.resolved_keyword_annotator import ResolvedKeywordAnnotator
from epitator.count_annotator import CountAnnotator
from epitator.date_annotator import DateAnnotator
from types import MethodType


In [6]:
def delete_non_epitator_name_entity_tiers(self):
    del self.tiers["spacy.nes"]
    del self.tiers["spacy.noun_chunks"]
    del self.tiers["spacy.sentences"]
    del self.tiers["spacy.tokens"]
    del self.tiers["nes"]
    del self.tiers["ngrams"]
    del self.tiers["tokens"]

In [7]:
def annotate(text, tiers=None):
    # Returns an annotated text
    doc = AnnoDoc(text)
    # Add bound method to delete spacy tiers. Necessary because pickling does not work otherwise
    doc.delete_non_epitator_name_entity_tiers = MethodType(delete_non_epitator_name_entity_tiers, doc)
    if tiers is None:
        doc.add_tiers(GeonameAnnotator())
        doc.add_tiers(ResolvedKeywordAnnotator())
        doc.add_tiers(CountAnnotator())
        doc.add_tiers(DateAnnotator())
    else:
        if not isinstance(tiers, list):
            tiers = [tiers]
        list(map(lambda tier: doc.add_tiers(eval(tier)), tiers))

    return doc

In [9]:
a = annotate('Lol, ich bin hier in Berlin und habe Ebola')

In [10]:
a.delete_non_epitator_name_entity_tiers()

In [12]:
a.tiers

{'geonames': AnnoTier([]),
 'resolved_keywords': AnnoTier([AnnoSpan(37-42, Ebola)['http://purl.obolibrary.org/obo/DOID_4325']]),
 'structured_data': AnnoTier([]),
 'structured_data.values': AnnoTier([]),
 'dates': AnnoTier([]),
 'raw_numbers': AnnoTier([]),
 'counts': AnnoTier([])}

In [ ]:
import pickle
pickle.dump(a, open('test.pkl', 'wb'))